In [17]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud_2020 #pip install git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
#import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

In [9]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_callout_culture = 'https://en.wikipedia.org/wiki/Call-out_culture'
callout_culture_save = 'wikipedia_call-out_culture.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

In [11]:
'''
Section 1:

Construct cells immediately below this that describe and download webcontent relating to your anticipated
final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial 
chunks of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas Dataframe.
'''

#Practice with wikipedia before trying on twitter later

#Scraping step 1 - request access and print/check chunk of output
wikiContentRequest = requests.get(wikipedia_callout_culture) #requesting access
print(wikiContentRequest.text[:2000]) #prints first 1000 lines (or words? not lines?)

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Call-out culture - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XhooOQpAMF0AAKolNv4AAAAE","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Call-out_culture","wgTitle":"Call-out culture","wgCurRevisionId":933911029,"wgRevisionId":933911029,"wgArticleId":55596031,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Use British English Oxford spelling from

In [12]:
#Scraping step 2 - convert/parse output from previous step from HTML using BeautifulSoup
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser') 
print(wikiContentSoup.text[:500]) 





Call-out culture - Wikipedia
document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XhooOQpAMF0AAKolNv4AAAAE","wgCSPNonce":!1,"wgCanonicalN


In [13]:
#Scraping step 3- there's still whitespace in output, so further steps needed to extract what we need
#Save HTML and inspect it
with open(callout_culture_save, mode='w', encoding='utf-8') as f: #what is this line doing, within the ()?
    f.write(wikiContentRequest.text)

In [15]:
#Scraping step 4a- extracting specific chunks
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:10]:
    print(pTag.text)

    #printing text of the first seven paragraphs; why are only three printing?
    #looks like there are only three paragraphs on the page total


Call-out culture (also known as outrage culture) is a form of public shaming that aims to hold individuals and groups accountable for their actions by calling attention to behavior that is perceived to be problematic, usually on social media.[1][2][3] A variant of the term, cancel culture, describes a form of boycott in which someone (usually a celebrity) who has shared a questionable or unpopular opinion, or has had  behavior in their past that is perceived to be either offensive or problematic called out on social media is "canceled"; they are completely boycotted by many of their followers or supporters, often leading to massive declines in celebrities' (almost always social media personalities) careers and fanbase.[4][5]

Michael Bérubé, a professor of literature at Pennsylvania State University, states, "in social media, what is known as 'callout culture' and 'ally theater' (in which people demonstrate their bona fides as allies of a vulnerable population) often produces a swell 

In [21]:
#Scraping step 4b- further refining chunk (consoliding the relevant part I want to process/ analyze)
contentParagraphs = []
for pTag in contentPTags:
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text)) #understand what this line is doing;
                                                #working with ALL paragraphs, not just first three

#converting to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                      paragraph-text
0  \nCall-out culture (also known as outrage cult...
1  Michael Bérubé, a professor of literature at P...
2  Lisa Nakamura, a professor at the University o...
3                                                 \n


In [ ]:
#Scraping step 5- extracting *meaningful* chunks - processing the text
#Using regex 

In [ ]:
'''
Section 2
Construct cells immediately below this that spider webcontent from another site with content 
relating to your anticipated final project. Specifically, identify urls on a core page, then
follow and extract content from them into a pandas Dataframe. In addition, demonstrate a recursive
spider, which follows more than one level of links (i.e., follows links from a site, then follows
links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that
you do not wander the web forever :-).
'''

In [22]:
otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://en.wikipedia.org/wiki/Social_media', 0, 'social media'), ('https://en.wikipedia.org/wiki/Boycott', 0, 'boycott'), ('https://en.wikipedia.org/wiki/Michael_B%C3%A9rub%C3%A9', 1, 'Michael Bérubé'), ('https://en.wikipedia.org/wiki/Pennsylvania_State_University', 1, 'Pennsylvania State University'), ('https://en.wikipedia.org/wiki/Lisa_Nakamura', 2, 'Lisa Nakamura'), ('https://en.wikipedia.org/wiki/Professor', 2, 'professor'), ('https://en.wikipedia.org/wiki/University_of_Michigan', 2, 'University of Michigan'), ('https://en.wikipedia.org/wiki/Livelihood', 2, 'livelihood')]


In [23]:
contentParagraphsDF['source'] = [wikipedia_callout_culture] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

,paragraph-text,source,paragraph-number
0,\nCall-out culture (also known as outrage cult...,https://en.wikipedia.org/wiki/Call-out_culture,0
1,"Michael Bérubé, a professor of literature at P...",https://en.wikipedia.org/wiki/Call-out_culture,1
2,"Lisa Nakamura, a professor at the University o...",https://en.wikipedia.org/wiki/Call-out_culture,2
3,\n,https://en.wikipedia.org/wiki/Call-out_culture,3


In [24]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

In [25]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,0,\nCall-out culture (also known as outrage cult...,https://en.wikipedia.org/wiki/Call-out_culture,None,None
1,1,"Michael Bérubé, a professor of literature at P...",https://en.wikipedia.org/wiki/Call-out_culture,None,None
2,2,"Lisa Nakamura, a professor at the University o...",https://en.wikipedia.org/wiki/Call-out_culture,None,None
3,3,\n,https://en.wikipedia.org/wiki/Call-out_culture,None,None
4,0,Social media are interactive computer-mediated...,https://en.wikipedia.org/wiki/Social_media,0,social media
...,...,...,...,...,...
140,2,"From 1996 to 2016, Bérubé edited ""Cultural Fro...",https://en.wikipedia.org/wiki/Michael_B%C3%A9r...,1,Michael Bérubé
141,3,Bérubé drew attention in the early 1990s for h...,https://en.wikipedia.org/wiki/Michael_B%C3%A9r...,1,Michael Bérubé
142,4,"Bérubé's third book, Life As We Know It: A Fat...",https://en.wikipedia.org/wiki/Michael_B%C3%A9r...,1,Michael Bérubé
143,5,"In 2005-06, Bérubé emerged as a critic of Davi...",https://en.wikipedia.org/wiki/Michael_B%C3%A9r...,1,Michael Bérubé


In [ ]:
'''
Section 3
Construct cells immediately below this that extract and organize textual content from text,
PDF or Word into a pandas dataframe.
'''

In [32]:
with open('../data/grammars/large_grammars/alvey_sentences.txt') as f:
    alvey = f.read()
print(alvey[200:300])

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf6 in position 358: invalid start byte

In [36]:
targetDir = '../data/grammars/large_grammars' #Change this to your own directory of texts
largegrammarText = []
largegrammarFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path) as f:
        largegrammarText.append(f.read())
    largegrammarFileName.append(file.name)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf6 in position 319: invalid start byte

In [ ]:
#shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
#shakespear_df